## Cкачивание необходимых библиотек

In [ ]:
pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 88.7 MB/s eta 0:00:00


In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.3 MB/s eta 0:00:00


In [ ]:
pip install hdbscan

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 77.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.29-cp310-cp310-linux_x86_64.whl size=3542008 sha256=3abcd612ca66535b90ccaa739aa355b8eb684f76635d494c82f0b8df49a9240d
  Stored in directory: /root/.cache/pip/wheels/dc/52/e3/6c6b60b126b4d5c4370cb5ac071b82950f91649d62d72f7f56
Successfully built hdbscan


## Подключение библиотек

In [ ]:
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import hdbscan

In [ ]:
import torch
import numpy
from transformers import BertTokenizer, BertModel, BertConfig

In [ ]:
import hdbscan

In [ ]:
import faiss

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt

## Скачка данных

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data.csv', delimiter=',')

In [ ]:
df

In [ ]:
tags = ['skill', 'knowledge', 'unknown']
filleted_df = df[df['tag'].isin(tags)]

In [ ]:
filleted_df

In [ ]:
from_count = dict(filleted_df['simple_form'].value_counts())

In [ ]:
from_count

## Получение эмбеддингов

In [ ]:
class BertVectorizer():
    def __init__(self, model_name):
        self.model_name = model_name
        self.load_model(model_name)

    def load_model(self, model_name):
        print(f"Загрузка модели: {model_name}")
        config = BertConfig.from_pretrained(model_name,
                                            output_hidden_states=True,
                                            output_attentions=True,
                                            return_dict=True)
        print("tokenizer loading... ", end='')
        if 'uncased' in model_name:
            lower_case = True
        else:
            lower_case = False
        print(f"lower_case={lower_case} ... ", end='')
        self.tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=lower_case)
        print("DONE")

        print("model loading...", end='')
        self.model = BertModel.from_pretrained(model_name, config=config)
        print("DONE")

    def get_output(self, text):
        with torch.no_grad():
            #input_ids = torch.LongTensor([tokenizer.encode(text, add_special_tokens=True)])
            #input_ids = torch.tensor([self.tokenizer.encode(text, add_special_tokens=True)])
            input_ids = self.tokenizer(text, return_tensors='pt')
        output = self.model(**input_ids)
        return output

    def get_CLS_vector(self, text):
        output = self.get_output(text)
        with torch.no_grad():
            cls = output.last_hidden_state[0][0] # первый CSL токен
            return numpy.array(cls)

    def get_MEANWORD_vector(self, text):
        output = self.get_output(text)
        with torch.no_grad():
            return numpy.array(output.last_hidden_state.mean(1)[0])

# if name == "main":
#     name_model = 'DeepPavlov/rubert-base-cased'
#     vm = BertVectorizer(name_model)
#     print(vm.get_CLS_vector("мама мыла раму")[:10])

In [ ]:
embeding = BertVectorizer('/content/drive/MyDrive/bert-ft')

In [ ]:
def convert_vectors_to_2D(dictionary):
    # Соберите все векторы в матрицу
    vectors = numpy.array(dictionary[item] for item in dictionary])

    pca = PCA(n_components=2)

    transformed_vectors = pca.fit_transform(vectors)

    for i, item in enumerate(dictionary):
        item["embedding"] = transformed_vectors[i]

    return dictionary

In [ ]:
pca = PCA(n_components=2)

In [ ]:
lst_count = list(from_count.items())[:1000]

In [ ]:
skills = []
for skill, count in lst_count:
    dict_info_skill = {}
    dict_info_skill['phrase'] = skill
    dict_info_skill['embedding'] =  embeding.get_MEANWORD_vector(skill)
    dict_info_skill['count'] = count
    skills.append(dict_info_skill)

In [ ]:
skills = convert_vectors_to_2D(skills)

In [ ]:
skills

In [ ]:
embeddings = [skill['embedding'] for skill in skills]
embeddings = numpy.array(embeddings)
# pca = PCA(n_components=2)
# reduced_embeddings = pca.fit_transform(embeddings)
# index = faiss.IndexFlatL2(embeddings.shape[1])
# index.add(embeddings)
res = faiss.StandardGpuResources()
index = faiss.GpuIndexFlatL2(res, 2)  # d - размерность вашего вектора

# gpu_vectors = faiss.float_vector_to_array(embeddings)  # vectors - ваши векторы
index.add(embeddings)

chosen_skills = set()  # Множество для хранения выбранных скиллов

faiss_skills = []
dict_faiss = {}

for skill in skills:
    skill_prhase = skill['phrase']
    if skill_prhase not in chosen_skills:
        max_count = skill['count']
        max_count_skill = skill  # Переменная для хранения скилла с максимальным количеством
        k = 5  # Количество ближайших соседей
        # distances, indices = index.search(numpy.expand_dims(skill['embedding'], axis=0), k)
        query_gpu = numpy.ascontiguousarray(numpy.expand_dims(skill['embedding'], axis=0))
        distances, indices = index.search(query_gpu, k)

        # Вывод результатов
        # print("Ближайшие соседи:")
        for i in range(k):
            neighbor_index = indices[0][i]
            neighbor_skill = skills[neighbor_index]
            # print(neighbor_skill['phrase'], neighbor_skill['count'])
            if neighbor_skill['count'] > max_count:
                max_count = neighbor_skill['count']
                max_count_skill = neighbor_skill

        chosen_skills.add(max_count_skill['phrase'])

        # print('--', max_count_skill['phrase'])

        # dict_faiss[max_count_skill['phrase']] = {'embedding': max_count_skill['embedding'], 'count': max_count_skill['count']}
        # faiss_skills.append(max_count_skill)
        dict_faiss[max_count_skill['phrase']] = list(max_count_skill['embedding'])

    # distance = distances[0][i]
    # text = skill['phrase'][index]
    # print(f"Текст: {text}, Расстояние: {distance}")

In [ ]:
dict_faiss

## Кластеризация hdbscan

In [ ]:
import numpy as np

In [ ]:
dict_faiss = convert_vectors_to_2D(skills)

In [ ]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=3, metric='euclidean')

In [ ]:
embdeddings = np.array([v for k, v in dict_faiss.items()])

In [ ]:
cluster_labels = clusterer.fit_predict(embeddings)

In [ ]:
cluster_labels

In [ ]:
cluster_skills = clusterer.labels_
len(cluster_skills)

In [ ]:
idx_0 = []
for idx in range(len(data)):
    if data[idx] == 18:
        idx_0.append(idx)
idx_0

In [ ]:
# Вывод результатов
for label in set(cluster_labels):
    cluster_indices = np.where(cluster_labels == label)[0]
    print(f"Кластер {label}: {cluster_indices}")

Кластер 0: [204 241 253 359 434 724 754 845 854 948]
Кластер 1: [  1   8  12  16  20  21  29  32  40  41  44  51  56  58  64  65  67  73
  77  79  84  87  89  91 103 105 110 114 135 140 141 143 146 157 158 168
 183 184 202 212 226 236 252 265 270 272 275 301 303 312 316 321 330 337
 349 352 364 378 380 386 393 411 424 426 429 439 444 453 455 456 462 478
 498 512 521 528 539 549 552 555 556 559 574 598 599 600 604 621 624 629
 637 655 658 659 678 683 707 729 733 745 746 749 750 753 777 790 800 840
 858 875 876 880 900 903 921 930 932 940 943 951 962 965 970 972 984 987
 988]
Кластер 2: [ 22 657 758 846 911]
Кластер 3: [389 403 433 482 605 688]
Кластер 4: [ 83 131 333 470 895 960]
Кластер 5: [206 442 583 669 689]
Кластер 6: [407 487 573 721 776 783]
Кластер 7: [111 323 553 661 690 795 856 883]
Кластер 8: [ 46 283 475 841 842 896]
Кластер 9: [  2  59 127 149 155 198 218 377 502 510 514 531 738 797 974]
Кластер 10: [ 23 107 180 766 920]
Кластер 11: [ 13 148 152 160 169 174 199 200 205 245 

In [ ]:


# Предположим, что у вас есть результаты кластеризации и матрица схожести между эмбеддингами
# cluster_labels = ...
similarity_matrix = cosine_similarity(embeddings)

# Создаем граф
graph = nx.Graph()

# Добавляем вершины кластеров в граф
for label in numpy.unique(cluster_labels):
    cluster_indices = numpy.where(cluster_labels == label)[0]
    graph.add_node(label, members=cluster_indices)

# Соединяем кластеры на основе схожести
num_clusters = len(numpy.unique(cluster_labels))
for i in range(num_clusters):
    for j in range(i+1, num_clusters):
        if graph.has_node(i) and graph.has_node(j):
            cluster_i_indices = graph.nodes[i]['members']
            cluster_j_indices = graph.nodes[j]['members']
            similarity = numpy.mean(similarity_matrix[cluster_i_indices][:, cluster_j_indices])
            graph.add_edge(i, j, weight=similarity)


# Вывод информации о графе
print("Количество вершин в графе:", graph.number_of_nodes())
print("Количество ребер в графе:", graph.number_of_edges())

# Дополнительные операции с графом
# ...



Количество вершин в графе: 28
Количество ребер в графе: 351


In [ ]:
import matplotlib.pyplot as plt

# Предположим, что у вас есть граф graph

# Создаем пустой граф для визуализации
visualization_graph = nx.Graph()

# Копируем узлы и ребра из оригинального графа в граф для визуализации
visualization_graph.add_nodes_from(graph.nodes())
visualization_graph.add_edges_from(graph.edges())

# Визуализируем граф
pos = nx.spring_layout(visualization_graph)
nx.draw(visualization_graph, pos=pos, with_labels=True, node_color='lightblue', edge_color='gray')

# Показываем граф
plt.show()


In [ ]:
from collections import defaultdict
import networkx as nx
import plotly.graph_objects as go
from typing import Any, List, Dict, Tuple, Union, Callable

Vertex = Any
Edge = Tuple[Vertex, Vertex]
Num = Union[int, float]


class GraphVisualization:
    def __init__(
        self,
        G: nx.Graph,
        pos: Dict[Vertex, Union[Tuple[Num, Num], Tuple[Num, Num, Num]]],
        node_text: Union[Dict[Vertex, str], Callable] = None,
        node_text_position: Union[Dict[Vertex, str], Callable, str] = None,
        node_text_font_color: Union[Dict[Vertex, str], Callable, str] = None,
        node_text_font_family: Union[Dict[Vertex, str], Callable, str] = None,
        node_text_font_size: Union[Dict[Vertex, Num], Callable, str] = None,
        node_size: Union[Dict[Vertex, Num], Callable, Num] = None,
        node_color: Union[Dict[Vertex, Union[str, Num]], Callable, Union[str, Num]] = None,
        node_border_width: Union[Dict[Vertex, Num], Callable, Num] = None,
        node_border_color: Union[Dict[Vertex, str], Callable, str] = None,
        node_opacity: Num = None,
        edge_width: Union[Dict[Edge, Num], Callable, Num] = None,
        edge_color: Union[Dict[Edge, str], Callable, str] = None,
        edge_opacity: Num = None,
    ):
        # check dimensions
        if all(len(pos.get(v, [])) == 2 for v in G):
            self.is_3d = False
        elif all(len(pos.get(v, [])) == 3 for v in G):
            self.is_3d = True
        else:
            raise ValueError

        # default settings
        self.default_settings = dict(
            node_text=str,  # show node label
            node_text_position="middle center",
            node_text_font_color='#000000',
            node_text_font_family='Arial',
            node_text_font_size=14,
            node_size=10 if self.is_3d else 18,
            node_color='#fcfcfc',
            node_border_width=2,
            node_border_color='#333333',
            node_opacity=0.8,
            edge_width=4 if self.is_3d else 2,
            edge_color='#808080',
            edge_opacity=0.8,
        )

        # save settings
        self.G = G
        self.pos = pos
        self.node_text = node_text
        self.node_text_position = node_text_position
        self.node_text_font_color = node_text_font_color
        self.node_text_font_family = node_text_font_family
        self.node_text_font_size = node_text_font_size
        self.node_size = node_size
        self.node_color = node_color
        self.node_border_width = node_border_width
        self.node_border_color = node_border_color
        self.node_opacity = node_opacity
        self.edge_width = edge_width
        self.edge_color = edge_color
        self.edge_opacity = edge_opacity

    def _get_edge_traces(self) -> List[Union[go.Scatter, go.Scatter3d]]:
        # group all edges by (color, width)
        groups = defaultdict(list)

        for edge in self.G.edges():
            color = self._get_setting('edge_color', edge)
            width = self._get_setting('edge_width', edge)
            groups[(color, width)] += [edge]

        # process each group
        traces = []
        for (color, width), edges in groups.items():
            x, y, z = [], [], []
            for v, u in edges:
                x += [self.pos[v][0], self.pos[u][0], None]
                y += [self.pos[v][1], self.pos[u][1], None]
                if self.is_3d:
                    z += [self.pos[v][2], self.pos[u][2], None]

            params = dict(
                x=x,
                y=y,
                mode='lines',
                hoverinfo='none',
                line=dict(color=color, width=width),
                opacity=self._get_setting('edge_opacity'),
            )

            traces += [go.Scatter3d(z=z, **params) if self.is_3d else go.Scatter(**params)]

        return traces

    def _get_node_trace(self, showlabel, colorscale, showscale, colorbar_title, reversescale) -> Union[go.Scatter, go.Scatter3d]:
        x, y, z = [], [], []
        for v in self.G.nodes():
            x += [self.pos[v][0]]
            y += [self.pos[v][1]]
            if self.is_3d:
                z += [self.pos[v][2]]

        params = dict(
            x=x,
            y=y,
            mode='markers' + ('+text' if showlabel else ''),
            hoverinfo='text',
            marker=dict(
                showscale=showscale,
                colorscale=colorscale,
                reversescale=reversescale,
                color=self._get_setting('node_color'),
                size=self._get_setting('node_size'),
                line_width=self._get_setting('node_border_width'),
                line_color=self._get_setting('node_border_color'),
                colorbar=dict(
                    thickness=15,
                    title=colorbar_title,
                    xanchor='left',
                    titleside='right'
                ),
            ),
            text=self._get_setting('node_text'),
            textfont=dict(
                color=self._get_setting('node_text_font_color'),
                family=self._get_setting('node_text_font_family'),
                size=self._get_setting('node_text_font_size')
            ),
            textposition=self._get_setting('node_text_position'),
            opacity=self._get_setting('node_opacity'),
        )

        trace = go.Scatter3d(z=z, **params) if self.is_3d else go.Scatter(**params)
        return trace

    def _get_setting(self, setting_name, edge=None):
        default_setting = self.default_settings.get(setting_name)
        def_func = default_setting if callable(default_setting) else lambda x: default_setting
        setting = self.__dict__.get(setting_name)

        if edge is None:  # vertex-specific
            if setting is None:  # default is used
                if callable(default_setting):  # default is a function
                    return [def_func(v) for v in self.G.nodes()]
                else:  # default is a constant
                    return default_setting
            elif callable(setting):  # setting is a function
                return [setting(v) for v in self.G.nodes()]
            elif isinstance(setting, dict):  # setting is a dict
                return [setting.get(v, def_func(v)) for v in self.G.nodes()]
            else:  # setting is a constant
                return setting
        else:  # edge-specific
            if setting is None:  # default is used
                return def_func(edge)
            elif callable(setting):  # setting is a function
                return setting(edge)
            elif isinstance(setting, dict):  # setting is a dict
                return setting.get(edge, def_func(edge))
            else:  # setting is a constant
                return setting

    def create_figure(
        self,
        showlabel=True,
        colorscale='YlGnBu',
        showscale=False,
        colorbar_title='',
        reversescale=False,
        **params
    ) -> go.Figure:
        axis_settings = dict(
            autorange=True,
            showgrid=False,
            zeroline=False,
            showline=False,
            visible=False,
            ticks='',
            showticklabels=False,
        )
        scene = dict(
            xaxis=axis_settings,
            yaxis=axis_settings,
            zaxis=axis_settings,
        )

        layout_params = dict(
            paper_bgcolor='rgba(255,255,255,255)',  # white
            plot_bgcolor='rgba(0,0,0,0)',  # transparent
            autosize=False,
            height=400,
            width=450 if showscale else 375,
            title='',
            titlefont_size=16,
            showlegend=False,
            hovermode='closest',
            margin=dict(b=5, l=0, r=0, t=20),
            annotations=[],
            xaxis=axis_settings,
            yaxis=axis_settings,
            scene=scene,
        )

        # override with the given parameters
        layout_params.update(params)

        # create figure
        fig = go.Figure(layout=go.Layout(**layout_params))
        fig.add_traces(self._get_edge_traces())
        fig.add_trace(self._get_node_trace(showlabel, colorscale, showscale, colorbar_title, reversescale))
        return fig

In [ ]:
dict_cluster_skills = {}
for skill in range(len(cluster_skills)):
    if cluster_skills[skill] in dict_cluster_skills:
        dict_cluster_skills[cluster_skills[skill]] += [skills[skill]]
    else:
        dict_cluster_skills[cluster_skills[skill]] = [skills[skill]]

dict_cluster_skills

In [ ]:
vis = GraphVisualization(graph, dict_faiss)
fig = vis.create_figure()
fig.show()

In [ ]:
import networkx as nx
import numpy as np
from sklearn.cluster import KMeans
import plotly.graph_objects as go


# Extract coordinates from the dictionary
coordinates = list(dict_faiss.values())

# Perform k-means clustering
kmeans = KMeans(n_clusters=30, random_state=0)
clusters = kmeans.fit_predict(coordinates)
print(clusters)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



[ 2  2 12 12 10 24 19 20  2 19  2  2  2 29 19 25 25  2  6 21  3 22 26  3
 14  0 12 27 19  1  3  9 24 20 23  6  6  3  4 27  5  4 12  0  5  5  4  8
  5 19  6  3  7  1 16 22  2  1 20 10 21 18  9 18 27 22 23 22 20 26 22  5
  6  9 19 14 24  4 23 28  4 21  8 11  5  6  4 21 27 16 27  0 17 24 13  9
 13 23  1  9 16 23 23  7 12 28  8  3  2 29 12 23  5 19 12 12 24 20 18  7
 23 24 19 14 29  9 24  5 19 10 21 29 12  7 10 13 29  0  9  8  7 13 10 12
 28 20  6  3  2  9 21  1  8  4  2 12  7 13 20 19  9 26 21 15  5  1 13  2
 28  6  8  3  4  9 14 23 12 10 21  5  3 29 23  8 20 15  8 16 28 20  5  7
  5 14 19 22  0 19 15 15  9 15 27 24 29 17 12 24  5 11  0  6 19 16 15 11
 14 17 10 26 24  4 15 16 25 22  9  4 21 13  4  6  5  8  5  8 23 16 18 14
 14 16  1 23  4 10 11  9 22 27  3  8  3 28 29  7 22 14 16 26 28 20 14  2
 19  8 28 28  7 24  4 23 15 20 19 16 11 21 24 19 23  3  6  6  6 17 12  0
 24  1  3 29 28 17 28 19 10 19 10 10 29 11 21 22 21 19 29  6  8 12 21 26
 22  4  7  6 27 20]


In [ ]:
for label in set(cluster_labels):
    cluster_indices = np.where(cluster_labels == label)[0]
    print(f"Кластер {label}: {cluster_indices}")

In [ ]:


# Create a graph
G = nx.Graph()

# Add nodes to the graph
for i, node in enumerate(dict_faiss.keys()):
    G.add_node(node, cluster=clusters[i])

# Add edges based on cluster membership
for i, node1 in enumerate(dict_faiss.keys()):
    for j, node2 in enumerate(dict_faiss.keys()):
        if clusters[i] == clusters[j] and i != j:
            G.add_edge(node1, node2)

# Generate the positions of the nodes using the spring layout algorithm
pos = nx.spring_layout(G, iterations=20)

vis = GraphVisualization(G, pos, node_size=10, node_border_width=2, edge_width=1)
fig = vis.create_figure(height=1000, width=1000, showlabel=True)
fig.show()


In [ ]:


# Create a graph
G = nx.Graph()

# Add nodes to the graph
for i, node in enumerate(dict_faiss.keys()):
    G.add_node(node, cluster=cluster_labels[i])

# Add edges based on cluster membership
for i, node1 in enumerate(dict_faiss.keys()):
    for j, node2 in enumerate(dict_faiss.keys()):
        if cluster_labels[i] == cluster_labels[j] and i != j:
            G.add_edge(node1, node2)

# Generate the positions of the nodes using the spring layout algorithm
pos = nx.spring_layout(G, iterations=20)

vis = GraphVisualization(G, pos, node_size=10, node_border_width=2, edge_width=1)
fig = vis.create_figure(height=1000, width=1000, showlabel=True)
fig.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go


# Извлечение данных для визуализации
phrases = list(dict_faiss.keys())
embeddings = np.array([item["embedding"] for item in dict_faiss.values()])
weights = np.array([item["count"] for item in dict_faiss.values()])

# Создание списков координат X и Y для вершин графа
x_coords = embeddings[:, 0]
y_coords = embeddings[:, 1]

# Создание списка ребер графа с учетом весов
# Create a list of graph edges with normalized weights
edges = []
max_weight = np.max(weights)
for i in range(len(phrases) - 1):
    for j in range(i + 1, len(phrases)):
        edge_weight = min(weights[i], weights[j]) / max_weight
        edges.append((i, j, edge_weight))

# Create the graph using Plotly
fig = go.Figure()

# Add vertices to the graph
fig.add_trace(go.Scatter(
    x=x_coords,
    y=y_coords,
    mode='markers',
    marker=dict(size=10),
    text=phrases,
    hoverinfo='text'
))

# Add edges to the graph with opacity based on weights
for edge in edges:
    i, j, edge_weight = edge
    x0, y0 = x_coords[i], y_coords[i]
    x1, y1 = x_coords[j], y_coords[j]
    fig.add_trace(go.Scatter(
        x=[x0, x1],
        y=[y0, y1],
        mode='lines',
        line=dict(color='gray', width=edge_weight, opacity=edge_weight),
    ))

# Configure the appearance of the graph
fig.update_layout(
    title='Graph of Embeddings with Weights',
    showlegend=False,
    hovermode='closest',
    width=600,
    height=600
)

# Display the graph
fig.show()

TypeError: ignored

In [ ]:
len(dict_faiss)

In [ ]:
test_df = df['simple_form'][:1000]

for idx in idx_0:
    print(test_df[idx])

In [ ]:
dict_claster = {}
for el in cluster_skills:
    if el in dict_claster:
        dict_claster[el] += 1
    else:
        dict_claster[el] = 1
dict_claster

In [ ]:
import numpy as np

In [ ]:
# vectors = []
# count_skills = 0
# for element_cluster in dict_cluster_skills:
#     for dict_skill in dict_cluster_skills[element_cluster]:
#         embedding_skill = []
#         embedding_skill.append(dict_skill['embedding'])
#     vectors.append(embedding_skill)
#         # embedding_skill.append(embedding)
# vectors